# Данный нотбук делает следующее:

*   Обучение Yolov5 на кастомном датасете с гугл диска/robflow
*   Экспорт модели в кастомный ONNX через репозиторий yolov5 от RKNN
*   Конвертация ONNX модели в RKNN



### Настройка окружений

In [ ]:
!apt install python3.10-venv

In [ ]:
!pip3 install ultralytics -U

### Копирование датасета с гугл диска

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content
!cp /content/drive/MyDrive/NozzleData/nozzle_v.0.2.1.zip .
!unzip nozzle_v.0.2.1.zip -d nozzle_data

In [5]:
!rm nozzle_v.0.2.1.zip

**В data.yaml указать абсолютные пути к директориям с картинками**

In [6]:
%cd /content

/content


In [7]:
import os
os.environ['WANDB_MODE'] = 'disabled'

### Обучение

In [ ]:
from ultralytics import YOLO


model = YOLO('yolov8n.pt')
model.train(data='/content/nozzle_data/data.yaml', epochs=25, device=0, batch=128, imgsz=640, augment=False)

### Копирование результатов на гугл диск

In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [14]:
!cp -r /content/runs/detect/train /content/drive/MyDrive

### Конвертация в ONNX

#### Установка

In [ ]:
!git clone https://github.com/airockchip/ultralytics_yolov8
%cd ultralytics_yolov8
!python3 -m venv env
!source env/bin/activate
!pip3 uninstall ultralytics -y

In [ ]:
!pip3 install onnx onnxruntime

#### Экспорт

In [ ]:
!PYTHONPATH=./ python ./ultralytics/engine/exporter.py

In [48]:
!cp /content/runs/detect/train/weights/best.onnx /content/drive/MyDrive

### Экспорт в RKNN

#### Установка

In [ ]:
%cd /content
!python3 -m venv env
!source env/bin/activate
!wget https://raw.githubusercontent.com/airockchip/rknn-toolkit2/refs/heads/master/rknn-toolkit2/packages/rknn_toolkit2-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip3 install rknn_toolkit2-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

In [7]:
import rknn

In [ ]:
!git clone https://github.com/airockchip/rknn_model_zoo
%cd rknn_model_zoo/examples/yolov8

In [14]:
import os

IMG_PATH = "/content/nozzle_data/train/images"
files = os.listdir(IMG_PATH)

with open("/content/data_subset.txt", "w") as fd:
  for i in files:
    print(os.path.join(IMG_PATH, i), file=fd)

In [16]:
%cd python

/content/rknn_model_zoo/examples/yolov8/python


In [ ]:
!python3 convert.py /content/runs/detect/train/weights/best.onnx rv1103 i8

I rknn-toolkit2 version: 2.2.0
--> Config model
done
--> Loading model
I Loading : 100%|██████████████████████████████████████████████| 126/126 [00:00<00:00, 26871.53it/s]
done
--> Building model
I Quantizating 6/11:  45%|██████████████████▌                      | 73/161 [00:16<00:05, 16.06it/s]